In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [0]:
!pip uninstall tensorflow

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [0]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 26kB/s 
     |████████████████████████████████| 3.8MB 37.4MB/s 
     |████████████████████████████████| 450kB 47.3MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import requests
import json

## Installing the transformers library
Let's start by installing the huggingface transformers library.

In [0]:
!pip install transformers

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

# Using BERT for text classification task

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/Fake_News_DB.db")
data = pd.read_sql("SELECT * from fake_news", conn)
data = data[2000:3740]

In [0]:
data.head(5)

,id,url,title,content,author,date,tags,type_of_article
2000,2000,https://panorama.pub/11646-znak-muzhchina-za-r...,В России могут ввести знак «Мужчина за рулём»,"В 2019 году на автомобилях, зарегистрированных...",Элла Нудельман,2018-12-22,Общество,fake
2001,2001,https://panorama.pub/11212-ombudsmen-po-delam-...,В России появится омбудсмен по делам «золотой ...,В России введут должность омбудсмена по делам ...,Борис Гонтермахер,2018-12-21,Общество,fake
2002,2002,https://panorama.pub/11698-kemeron-zasudit-ros...,Джеймс Кэмерон намерен засудить «Роскосмос» за...,Голливудский режиссер Джеймс Кэмерон решил под...,Соломон Жидков,2018-12-21,Общество,fake
2003,2003,https://panorama.pub/11565-traktor-dlya-davki....,В Белоруссии разработали специальный трактор д...,Необычный подарок к Новому году из Белоруссии ...,Ян Цмок,2018-12-20,Общество,fake
2004,2004,https://panorama.pub/11632-tolstogo-v-lono-tse...,Константинопольский патриархат вернул Льва Тол...,Решением Константинопольского патриархата под ...,Виталий Манн,2018-12-20,Общество,fake


In [0]:
data['type_of_article'].value_counts()

real    870
fake    870
Name: type_of_article, dtype: int64

In [0]:
y = {'real': 0, 'fake': 1}
data['y'] = data['type_of_article'].map(lambda x: y[x])

data_X = data['title']
data_y = data['y']

### Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [0]:
model_class, tokenizer_class, pretrained_weights = (ppb.TFDistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Теперь в переменной model содержится предобученная модель DistilBERT.

## Model #1: Preparing the Dataset
Перед тем, как передать предложения в BERT, нужно привести их к необходимому формату.

### Tokenization
Первый шаг - токенизация.

In [0]:
tokenized = data_X.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [0]:
print(tokenized[2000])

[101, 1182, 1195, 14150, 29747, 29747, 15414, 1191, 14150, 29741, 29748, 22919, 1182, 25529, 15290, 29747, 22919, 10325, 1187, 19865, 23925, 1077, 1191, 29748, 29743, 29752, 10325, 19865, 1187, 10260, 1195, 29748, 29436, 15290, 29745, 1090, 102]


### Padding
После токенизации, `tokenized` - это список предложений - каждое предложение представлено в виде списка токенов. Затем применяем паддинг, чтобы привести все предложения к одинаковой длине.

In [0]:
print('Max sentence length: ', max([len(sen) for sen in tokenized]))

Max sentence length:  130


In [0]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 150

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

padded = pad_sequences(tokenized, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 150 values...

Padding token: "[PAD]", ID: 0

Done.


In [0]:
padded = np.array(padded)
padded.shape

(1740, 150)

In [0]:
padded[0]

array([  101,  1182,  1195, 14150, 29747, 29747, 15414,  1191, 14150,
       29741, 29748, 22919,  1182, 25529, 15290, 29747, 22919, 10325,
        1187, 19865, 23925,  1077,  1191, 29748, 29743, 29752, 10325,
       19865,  1187, 10260,  1195, 29748, 29436, 15290, 29745,  1090,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

### Masking
Кроме переменной `padded` создадим другую переменную `attention_mask`, которая поможет игнорировать паддинг.

In [0]:
attention_masks = []

for sent in padded:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
attention_masks = np.array(attention_masks)   

attention_masks.shape

(1740, 150)

In [0]:
attention_masks[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [0]:
outputs = model(padded, attention_mask = attention_masks)

In [0]:
len(outputs)

1

In [0]:
last_hidden_states = outputs[0] 

In [0]:
last_hidden_states.shape

TensorShape([1740, 150, 768])

У нас есть:

1740 текстов;
150 - максимальное количество токенов на текст ([CLS] включен в этот список);
768 - размерность вектора BERT;
Теперь извлечем вектор [CLS] для каждого текста.

In [0]:
bert_features = last_hidden_states[:,0,:]
bert_features.shape

TensorShape([1740, 768])

In [0]:
bert_features = bert_features.numpy()

Метки, показывающие, является ли сообщение спамом записываем в переменную `lables`

In [0]:
labels = data_y

#### Train/Test Split
Разделим датасет на тренировочную и тестовую выборки.

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(bert_features, labels)

#### Logistic regression model

Обучим модель LogisticRegression.

In [0]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### Evaluation model 
Проверим, насколько хорошо модель справляется с задачей классификиции с помощью метрики accuracy:

In [0]:
lr_clf.score(test_features, test_labels)

0.9126436781609195

Также посмотрим на результаты dummy classifier:

In [0]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.496 (+/- 0.08)


In [0]:
from sklearn.metrics import classification_report
target_names = ['1', '0']
y_pred = lr_clf.predict(test_features)
print(classification_report(test_labels, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           1       0.94      0.89      0.91       226
           0       0.89      0.94      0.91       209

    accuracy                           0.91       435
   macro avg       0.91      0.91      0.91       435
weighted avg       0.91      0.91      0.91       435



In [0]:
lr_clf.predict(test_features[9].reshape(1, -1))

array([1])

In [0]:
pickle.dump(lr_clf, open('model.pkl','wb'))

In [0]:
model = pickle.load(open('model.pkl','rb'))
print(model.predict(test_features[0].reshape(1, -1)))

[1]


In [0]:
x = ['Алексей Навальный проведёт «Марш миллионов» в Twitter', 'Песков рассказал детали посещения Путиным больницы в Коммунарке', 'Безработные в Подмосковье будут получать региональную компенсацию']

In [0]:
tokenized_new = []
for sent in x:
    tokenized_new.append(tokenizer.encode(sent, add_special_tokens=True))

In [0]:
print(tokenized_new[0])

[101, 1180, 29436, 15290, 23925, 29747, 15290, 10325, 1192, 10260, 25529, 10260, 29436, 23742, 18947, 29113, 10325, 1194, 16856, 19259, 15290, 29742, 15290, 22919, 1077, 1191, 10260, 16856, 29753, 1191, 10325, 29436, 29436, 10325, 14150, 18947, 19259, 1090, 1182, 10474, 102]


In [0]:
padded_new = pad_sequences(tokenized_new, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

In [0]:
padded_new = np.array(padded_new)
padded_new.shape

(3, 150)

In [0]:
attention_masks_new = []

for sent in padded_new:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks_new.append(att_mask)
attention_masks_new = np.array(attention_masks_new)   

attention_masks_new.shape

(3, 150)

In [0]:
model_new = model_class.from_pretrained(pretrained_weights)

In [0]:
outputs_new = model_new(padded_new, attention_mask = attention_masks_new)

In [0]:
last_hidden_states_new = outputs_new[0] 

In [0]:
bert_features_new = last_hidden_states_new[:,0,:]

In [0]:
bert_features_new = bert_features_new.numpy()

In [0]:
bert_features_new.shape

(3, 768)

In [0]:
lr_clf.predict(bert_features_new[0].reshape(1, -1))

array([1])

In [0]:
lr_clf.predict(bert_features_new[1].reshape(1, -1))

array([1])

In [0]:
lr_clf.predict(bert_features_new[2].reshape(1, -1))

array([1])